In [2]:
# library
import numpy as np
import itertools
import requests
import polyline
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry


import warnings 

warnings.filterwarnings('ignore')

In [3]:
def calculate_straight_distance(lat1, lon1, lat2, lon2):

    km_constant = 3959* 1.609344
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    km = km_constant * c
    
    return km

In [4]:
# OSRM base #
#############
def get_res(point):

   status = 'defined'

   session = requests.Session()
   retry = Retry(connect=3, backoff_factor=0.5)
   adapter = HTTPAdapter(max_retries=retry)
   session.mount('http://', adapter)
   session.mount('https://', adapter)

   overview = '?overview=full'
   loc = f"{point[0]},{point[1]};{point[2]},{point[3]}" # lon, lat, lon, lat
   # url = "http://127.0.0.1:5000/route/v1/driving/"
   url = 'http://router.project-osrm.org/route/v1/driving/'
   r = session.get(url + loc + overview) 
   
   if r.status_code!= 200:
      
      status = 'undefined'
      
      # distance    
      distance = calculate_straight_distance(point[1], point[0], point[3], point[2]) * 1000
      
      # route
      route = [[point[0], point[1]], [point[2], point[3]]]

      # duration & timestamp
      speed_km = 10 # km/h
      speed = (speed_km * 1000/60)    # m/m  
      duration = distance/speed # 분
      
      timestamp = [0, duration]

      result = {'route': route, 'timestamp': timestamp, 'duration': duration, 'distance' : distance}
   
      return result, status
   
   res = r.json()   
   return res, status


In [5]:
# 서울 시청에서 강남역
point_city_hall_to_gangnam = [126.9780, 37.5665, 127.0281, 37.4980]  # [lon1, lat1, lon2, lat2]
result, status = get_res(point_city_hall_to_gangnam)

In [7]:
result

{'code': 'Ok',
 'routes': [{'geometry': '{hhdFgk_fWBfA?h@?\\q@?aC@k@?wC@QECCUe@?G@O?CBy@NyFB[@WDqA@q@NqEH{DDkA?SBk@Fm@\\gDd@sE^sCLoATyB@gBBoAr@@pBF`ADJATAp@Ad@AZAT?F?B?xA?`CCZGl@U??VKb@OJCZOLGl@Y\\YxA}@\\SZM\\KXElBOzAOD?ZAf@G\\ETERERK^Q`@a@b@i@`@cALm@PaA@E\\aAb@iAZg@j@m@b@a@h@Wh@Sz@KbAKdCSl@I\\E`@Iv@M|Bw@v@UtA{@jlA{o@PMZYv@m@LOv@u@f@cAd@yA`@iAZu@V_@RUh@]`@QREt@I`HiAlC_AXIx@Uj@O`Ag@|@i@jAcA~BwA|@m@`Ao@|@m@pA}@tBuApBsAr@g@r@i@hA{@^[r@]\\SROfC}AbAs@`As@vBiBhBgBdC{BvHaH~AyAvIeIrHyGlBiBd@c@v@k@`BgAr@SfAY~AQfJkBj@AXCpCg@vFcABA^GZGb@KnA[j@M\\IfAO~AY`AQd@IjCm@lA]bBo@pEcBHEHC|Ae@bE_BxB}@rAi@r@Wz@]dA_@zB{@LGbBo@xCiAjEaBf@Sh@MtCiAbCaAx@[rBq@pAe@nB{@t@YnEgBf@QXMdBm@n@UZMdAa@VIHEJECQAGCMUgA',
   'legs': [{'steps': [],
     'summary': '',
     'weight': 621.7,
     'duration': 621.7,
     'distance': 9994.8}],
   'weight_name': 'routability',
   'weight': 621.7,
   'duration': 621.7,
   'distance': 9994.8}],
 'waypoints': [{'hint': 'zaYOhf___38MAAAAIQAAAAAAAAAVAAAAuNeWQcbL_EEAAAAAAGr-QQwAAAAhAAAAAAA

In [9]:
##############################
# Extract duration, distance #
##############################
def extract_duration_distance(res):
       
   # duration = res['routes'][0]['duration']
   duration = res['routes'][0]['duration']/(30)
   distance = res['routes'][0]['distance']
   
   return duration, distance

In [11]:
duration, distance = extract_duration_distance(result)

In [12]:
duration

20.723333333333336

In [13]:
distance

9994.8

In [14]:
#################
# Extract route #
#################
def extract_route(res):
    
    route = polyline.decode(res['routes'][0]['geometry'])
    route = list(map(lambda data: [data[1],data[0]] ,route))
    
    return route

In [15]:
route = extract_route(result)

In [16]:
route

[[126.97796, 37.56702],
 [126.9776, 37.567],
 [126.97739, 37.567],
 [126.97724, 37.567],
 [126.97724, 37.56725],
 [126.97723, 37.5679],
 [126.97723, 37.56812],
 [126.97722, 37.56888],
 [126.97725, 37.56897],
 [126.97727, 37.56899],
 [126.97746, 37.5691],
 [126.9775, 37.5691],
 [126.97758, 37.56909],
 [126.9776, 37.56909],
 [126.97789, 37.56907],
 [126.97914, 37.56899],
 [126.97928, 37.56897],
 [126.9794, 37.56896],
 [126.97981, 37.56893],
 [126.98006, 37.56892],
 [126.98111, 37.56884],
 [126.98205, 37.56879],
 [126.98243, 37.56876],
 [126.98253, 37.56876],
 [126.98275, 37.56874],
 [126.98298, 37.5687],
 [126.98382, 37.56855],
 [126.98488, 37.56836],
 [126.98562, 37.5682],
 [126.98602, 37.56813],
 [126.98663, 37.56802],
 [126.98715, 37.56801],
 [126.98755, 37.56799],
 [126.98754, 37.56773],
 [126.9875, 37.56716],
 [126.98747, 37.56683],
 [126.98748, 37.56677],
 [126.98749, 37.56666],
 [126.9875, 37.56641],
 [126.98751, 37.56622],
 [126.98752, 37.56608],
 [126.98752, 37.56597],
 [126.987